## Homework 2 Part II (3 points + 1 Bonus)

__Welcome to the last homework!__

In this homework we take a look at the annotations you made in homework 2 part I. Each student annotated 100 paragraphs, each paragraph was annotated by three students. Now, we are interested in how reliable the annotations are. For that we can use a measure of reliability - coefficients of agreement. In the [lab](https://colab.research.google.com/drive/10bol3Mf38KKm_rvTc5F8Lv-wKF-pPK1E?usp=sharing) we used Cohen's Kappa, which can be used when there are only two annotators, which is not the case for us (reminder: we have three). Also, we have multi-label dataset, meaning that each paragraph have either no labels or one to many labels.

We are going to measure overall agreement and also agreement for each label separately. For overall agreement we are going to use Krippendorff's Alpha and for each label we are going to use Fleiss' Kappa. 




### Task 1. Download the data

Go to shared [Google Drive folder](https://drive.google.com/drive/folders/19bOcAuwhmIrH2dWWG985e1Ewz0wfJP-i?usp=sharing) and download file nlp_annotations.json. 

Data is in JSON format: 

{'annotations':[{ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'id':0, <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'annotation1': ["GUILT"], <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'annotation2': [], <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'annotation3': ["GUILT", "HARM"],<br>
&nbsp;&nbsp;&nbsp;},<br>
&nbsp;&nbsp;&nbsp;{<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'id':1, <br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'annotation1': ["CONCENTRATION", "INTERESTS", "ENERGY"],<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'annotation2': ["CONCENTRATION", "INTERESTS"],<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'annotation3': ["CONCENTRATION", "INTERESTS"]<br>
    
&nbsp;&nbsp;&nbsp;}<br>
&nbsp;&nbsp;&nbsp;...<br>
]}

As you can see, each instance has an id and annotations from 3 annotators. 

In [ ]:
!pip install nltk==3.6.2  # install the latest version of nltk

In [ ]:
import json

In [ ]:
with open('nlp_annotations.json', 'r', encoding='utf-8') as f: 
  data = json.load(f)

### Task 2. Calculate Krippendorff's Alpha (1p)

Krippendorff's Alpha $\alpha$ is a reliability coefficient that measures the agreement amoung observers, coders, judges, raters. It supports categorical, ordincal, interval and ratio type data. 

General form is 
$$\alpha = 1 - \frac{D_o}{D_e}$$
where $D_o$ is the observed disagreement among values assigned to units of analysis: 
$$D_o = \frac{1}{n}\sum_c\sum_ko_{ck metric}\delta^2_{ck}$$
and $D_e$ is the disagreement one would expect when the coding of units is attributable to change rahter than to the properties of these units: 
$$D_e = \frac{1}{n(n-1)}\sum_c\sum_kn_c*n_{k metric}\delta^2_{ck}$$
$o_{ck}, n_c, n_k, n$ refer to the frequencies of values in coincidence matrices. $_{metric}\delta^2_{ck}$ is a difference function.  

Ideally $\alpha=1$, which indicates perfect reliability amoung raters. You can see examples of the coincidence matrices and difference functions from [here](https://repository.upenn.edu/cgi/viewcontent.cgi?article=1043&context=asc_papers). 

[Measuring Agreement on Set-valued Items](http://www.cs.columbia.edu/nlp/papers/2006/passonneau_06.pdf) (MASI) is a distance metric used for comparing two sets. MASI becomes closer to 1 as sets in question have more members in common and are more equal in size, and closer to 0 when the sets are disjoint. MASI distance can be calculated: 

$$MASI = 1 - J*M$$
Here $J$ is the Jaccard metric for comparing two sets. It is a ratio of the cardinality of the intersection of the sets to their union (length of the intersection of the sets divided by the length of the union of sets). If two sets are equal, then $J=1$, if two sets are disjoint $J=0$.  $M$ (for monotonicity) is a four-point scale, $M=1$ when sets are identical, $M=\frac{2}{3}$ when one is a subset of the other, $M=\frac{1}{3}$ when the intersection of the sets is non-null and $M=0$ when the sets are disjoint. 


You can use [nltk.metrics.agreement.AnnotationTask](https://www.nltk.org/api/nltk.metrics.html) to calculate Krippendorff's Alpha using MASI distance. You need to specify the correct distance metric and convert the data into a format that AnnotationTask accepts. Read the documentation to understand the format. 

In [ ]:
from nltk.metrics.agreement import AnnotationTask
from nltk.metrics import masi_distance

In [ ]:
tuples = []

        
task = ...
alpha = ...
print(alpha)

### Task 3. Calculate Fleiss' Kappa (1p)

Calculate [Fleiss's Kappa](https://en.wikipedia.org/wiki/Fleiss%27_kappa#:~:text=Fleiss%27%20kappa%20(named%20after%20Joseph,of%20items%20or%20classifying%20items.) for each label. For that you have to iterate over all the labels, create new annotations (label is or is not assigned) for each label and calculate Fleiss' Kappa on these annotations. 

For example, if you have label 'MOOD', you iterate over all the examples in the dataset and check if the annotator had assigned this label 'MOOD' to the instance or not. If yes, then add new label as 1, if not, the new label would be 0 (the new labels can be anything, they do not have to be 0 and 1). When we have annotations: 
{"id": "9", "annotation1": [], "annotation2": ["CONFIDENCE"], "annotation3": ["MOOD"]}, {"id": "10", "annotation1": ["MOOD", "INTERESTS", "CONFIDENCE", "HARM", "GUILT"], "annotation2": [], "annotation3": ["MOOD", "CONFIDENCE"]}

We get tuples as (rater, id, annotation):


[('1', '9', 0), # MOOD not present<br>
 ('2', '9', 0), # MOOD not present <br>
 ('3', '9', 1), # MOOD present<br>
 ('1', '10', 1), # MOOD present<br>
 ('2', '10', 0), # MOOD not present<br>
 ('3', '10', 1), #MOOD present <br>



Formula for Fleiss's Kappa: 
$$\kappa=\frac{\overline{P}-\overline{P}_e}{1-\overline{P}_e}$$
$1-\overline{P}_e$ is agreement attainable above chance and $\overline{P}-\overline{P}_e$ is agreement actually achieved above chance. If all of the raters are in complete agreement then $\kappa$ is 1 and if there is no agreement then $\kappa$ is 0. Overline over $P$ and $P_e$ denote the mean of $P$ and $P_e$ respectively. 

You can use nltk.metrics.agreement.AnnotationTask again here. 



In [ ]:
labels = ["MOOD", "INTERESTS", "ENERGY", "SLEEP", "CONCENTRATION", "CONFIDENCE", "APPETITE", "HARM", "MOVEMENTS", "GUILT"]
kappas =  {'MOOD':0, 'INTERESTS':0, 'ENERGY':0, 'SLEEP':0, 'CONCENTRATION':0, 'CONFIDENCE':0, 'APPETITE':0, 'HARM':0, 'MOVEMENTS':0, 'GUILT':0}
tuples = []
for label in labels:
  # iterate over the whole dataset and fill in the tuples
  ...

  # initialize AnnotationTask
  ....

  # get the Fleiss' Kappa value
  kappa = ...
  kappas[label] = kappa
  tuples = []



__Test the answer__

In [ ]:
correct_alpha = 0.23219128493511998
correct_kappas = {'MOOD': 0.22540873401820205, 'INTERESTS': 0.2850924889362735, 'ENERGY': 0.36140539747008826, 'SLEEP': 0.4356703871206034, 'CONCENTRATION': 0.1706661261740384, 'CONFIDENCE': 0.2582967036519187, 'APPETITE': 0.5173003927419546, 'HARM': 0.5492254660481732, 'MOVEMENTS': 0.12092089475997503, 'GUILT': 0.22703412152247923}
assert correct_alpha == alpha, 'Correct Krippendorff\'s Alpha is {}, check your calculations.'.format(correct_alpha)
assert correct_kappas == kappas, 'Something wrong with these {}'.format([label for label in labels if correct_kappas[label] != kappas[label]])

### Task 4. Questions (1p)

Look at the values you got from Krippendorff's Alpha and Fleiss' Kappa. What can you say about these values? 

Answer: 



### BONUS. (1p)

Calculate Krippendorff's Alpha and Fleiss' Kappa ratings on the subset that you annotated. You can find what range of indices you annotated from the file students_ids.txt. You can find it in the shared [Google Drive folder](https://drive.google.com/drive/folders/19bOcAuwhmIrH2dWWG985e1Ewz0wfJP-i?usp=sharing) . Calculate the Krippendorff's Alpha and Fleiss's Kappa on this subset. If you did not annotate this dataset, you can use a random subset of the dataset or use a range specified for somebody else. 

Question: What is the range of indices you annotated (or chose)?

Answer: 

In [ ]:
subset = ...

subset_alpha = ...

In [ ]:
subset_kappas =  {'MOOD':0, 'INTERESTS':0, 'ENERGY':0, 'SLEEP':0, 'CONCENTRATION':0, 'CONFIDENCE':0, 'APPETITE':0, 'HARM':0, 'MOVEMENTS':0, 'GUILT':0}
tuples = []
for label in labels:
  # iterate over the subset of the dataset and fill in the tuples
  ...

  # initialize AnnotationTask
  ....

  # get the Fleiss' Kappa value
  kappa = ...
  subset_kappas[label] = kappa
  tuples = []


Look at the values you got. What can you say about them? 

Answer: 